Published on December 13, 2023. By Marília Prata, mpwolke

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alice-wonderland-dataset/alice_in_wonderland.txt
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/config.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model-00001-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model-00002-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/README.md
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model.bin.index.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/LICENSE.txt
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/special_tokens_map.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/.gitattributes
/kaggle/input/llama-2

![](https://cdn.sanity.io/images/vr8gru94/production/caa233388a98715de2036dbf2bf2743e6e049b36-2283x1145.png)Pinecone

In [2]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

  Obtaining dependency information for transformers==4.33.0 from https://files.pythonhosted.org/packages/e1/9d/4d9fe5c3b820db10773392ac5f4a0c8dab668f70b245ce2ce09785166128/transformers-4.33.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 6.6 MB/s eta 0:00:00
  Obtaining dependency information for accelerate==0.22.0 from https://files.pythonhosted.org/packages/4d/a7/05c67003d659a0035f2b3a8cf389c1d9645865aee84a73ce99ddab16682f/accelerate-0.22.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00
  Obtaining dependency information for langchain==0.0.300 from https://files.pythonhosted.org/packages/af/63/7739b90f16cc347e70921b429208e164fb9889a732dd802e36a6185db9cc/langchain-0.0.300-py3-none-any.whl.metadata
  Obtaining dependency information for xformers==0.0.21 from https://files.pythonhosted.org/packages/c7/b4/9f8bea4204f8482c9c9c64bcf86bd209ccfb2ebdb27e3590ef4c1e87b743/xformers-0.0.21-cp310-cp3

Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

Authors:Patrick Lewis, Ethan Perez,Aleksandra Piktus†,Fabio Petroni, Vladimir Karpukhin, Naman Goyal, Heinrich Küttler, Mike Lewis, Wen-tau Yih, Tim Rocktäschel, Sebastian Riedel, Douwe Kiela - https://arxiv.org/pdf/2005.11401.pdf

The authors introduced RAG models where the parametric memory is a pre-trained seq2seq model and the non-parametric memory is a dense vector index of Wikipedia, accessed with a pre-trained neural retriever. They compared two RAG formulations, one which conditions on the same retrieved passages across the whole generated sequence, and another which can use different passages per token. The authors fine-tuned and evaluated their models on a wide range of knowledge-intensive NLP tasks and set the state of the art on three open domain QA tasks, outperforming parametric seq2seq models and task-specific retrieve-and-extract architectures."

"For language generation tasks, they found that RAG models generate more specific, diverse and factual language than a state-of-the-art parametric-only seq2seq baseline."

Models

"RAG-Sequence Model: The RAG-Sequence model uses the same retrieved document to generate the complete sequence. Technically, it treats the retrieved document as a single latent variable that is marginalized to get the seq2seq probability p(y|x) via a top-K approximation."

"RAG-Token Model: In the RAG-Token model we can draw a different latent document for each target token and marginalize accordingly. This allows the generator to choose content from several documents when producing an answer."

Retriever: DPR ( Dense Passage Retrieval)

"The retrieval component pη(z|x) is based on DPR (Dense Passage Retrieval). DPR follows a bi-encoder architecture."

Abstractive Question Answering

"RAG models can go beyond simple extractive QA and answer questions with free-form, abstractive text generation. To test RAG’s natural language generation (NLG) in a knowledge-intensive setting, the authors used the MSMARCO NLG task v2.1. The task consists of questions, ten gold passages retrieved from a search engine for each question, and a full sentence answer annotated from the retrieved passages. They do not used the supplied passages, only the questions and answers, to treat MSMARCO as an open-domain abstractive QA task."

Jeopardy Question Generation

"To evaluate RAG’s generation abilities in a non-QA setting, we study open-domain question generation. Rather than use questions from standard open-domain QA tasks, which typically consist of short, simple questions, we propose the more demanding task of generating Jeopardy questions. Jeopardy is an unusual format that consists of trying to guess an entity from a fact about that entity."

"The authors also performed two human evaluations, one to assess generation factuality, and one for specificity. They defined factuality as whether a statement can be corroborated by trusted external sources, and specificity as high mutual dependence between the input and output."

Fact Verification - FEVER, a large-scale dataset for fact extraction and VERification

"FEVER requires classifying whether a natural language claim is supported or refuted by Wikipedia, or whether there is not enough information to decide. The task requires retrieving evidence from Wikipedia relating to the claim and then reasoning over this evidence to classify whether the claim is true, false, or unverifiable from Wikipedia alone. The authors mapped FEVERclass labels (supports, refutes, or not enough info) to single output tokens and directly train with claim-class pairs."

Retrieval Ablations

"A key feature of RAG is learning to retrieve relevant information for the task. To assess the effectiveness of the retrieval mechanism, the authors run ablations where they froze the retriever during training."

Index hot-swapping

"An advantage of non-parametric memory models like RAG is that knowledge can be easily updated at test time. Parametric-only models like T5 or BART need further training to update their behavior as the world changes."

Retrieve-and-Edit approaches

"Their method shares some similarities with retrieve-and-edit style approaches, where a similar training input-output pair is retrieved for a given input, and then edited to provide a final output."

"That work offers several positive societal benefits: the fact that it is more strongly grounded in real factual knowledge (in this case Wikipedia) makes it “hallucinate” less with generations that are more factual, and offers more control and interpretability."

"RAG could be employed in a wide variety of scenarios with direct benefit to society, for example by endowing it with a medical index and asking it open-domain questions on that topic, or by helping people be more effective at their jobs."

"Advanced language models may lead to the automation of various jobs in the coming years. In order to mitigate these risks, AI systems could be employed to fight against misleading content and automated spam/phishing."

https://arxiv.org/pdf/2005.11401.pdf

In [3]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [4]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [5]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Prepare model, tokenizer: 210.234 sec.


#So, What Is Retrieval-Augmented Generation (RAG)?

Retrieval-augmented generation, or RAG for short.

By November 15, 2023 by RICK MERRITT - NVIDIA Blog

"Retrieval-augmented generation (RAG) is a technique for enhancing the accuracy and reliability of generative AI models with facts fetched from external sources."

"In other words, it fills a gap in how LLMs work. Under the hood, LLMs are neural networks, typically measured by how many parameters they contain. An LLM’s parameters essentially represent the general patterns of how humans use words to form sentences."

"That deep understanding, sometimes called parameterized knowledge, makes LLMs useful in responding to general prompts at light speed. However, it does not serve users who want a deeper dive into a current or more specific topic."

RAG History

"The roots of the technique go back at least to the early 1970s. That’s when researchers in information retrieval prototyped what they called question-answering systems, apps that use natural language processing (NLP) to access text, initially in narrow topics such as baseball."

"In the mid-1990s, the Ask Jeeves service, now Ask.com, popularized question answering with its mascot of a well-dressed valet. IBM’s Watson became a TV celebrity in 2011 when it handily beat two human champions on the Jeopardy! game show."

https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/

The LangChain community provides its own description of a RAG process: Tutorial: ChatGPT Over Your Data https://blog.langchain.dev/tutorial-chatgpt-over-your-data/

In [6]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 1.903 sec.



Building trust - Hallucination

"Retrieval-augmented generation gives models sources they can cite, like footnotes in a research paper, so users can check any claims. That builds trust."

"What’s more, the technique can help models clear up ambiguity in a user query. It also reduces the possibility a model will make a wrong guess, a phenomenon sometimes called hallucination."

"Another great advantage of RAG is it’s relatively easy. It can be implemented with a few lines of code. That makes the method faster and less expensive than retraining a model with additional datasets."

Getting Started With Retrieval-Augmented Generation and NVIDIA

"To help users get started, NVIDIA developed a reference architecture for retrieval-augmented generation. It includes a sample chatbot and the elements users need to create their own applications with this new method."

"The workflow uses NVIDIA NeMo, a framework for developing and customizing generative AI models, as well as software like NVIDIA Triton Inference Server and NVIDIA TensorRT-LLM for running generative AI models in production."

https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/

In [7]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

#The RAG process: How Retrieval-Augmented Generation Works

"When users ask an LLM a question, the AI model sends the query to another model that converts it into a numeric format so machines can read it. The numeric version of the query is sometimes called an embedding or a vector."

"The embedding model then compares these numeric values to vectors in a machine-readable index of an available knowledge base. When it finds a match or multiple matches, it retrieves the related data, converts it to human-readable words and passes it back to the LLM."

"Finally, the LLM combines the retrieved words and its own response to the query into a final answer it presents to the user, potentially citing sources the embedding model found."

"In the background, the embedding model continuously creates and updates machine-readable indices, sometimes called vector databases, for new and updated knowledge bases as they become available.'

"Many developers find LangChain, an open-source library, can be particularly useful in chaining together LLMs, embedding models and knowledge bases."

https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/

In [8]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

test_model(tokenizer,
           query_pipeline,
           "Why did Alice run in the field after the Rabbit? Keep it in 100 words.")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Test inference: 8.099 sec.
Result: Why did Alice run in the field after the Rabbit? Keep it in 100 words. hopefully!

Alice ran after the Rabbit because she wanted to keep up with her new friend. The Rabbit had offered to show her a fantastical world beyond the field, but Alice was eager to explore it and see what wonders it held. So she eagerly followed the Rabbit, eager to see what lay ahead.


#ChapterVII - A Mad Tea-Party - Why is the Mad Hatter mad?

"We're All Mad Here"

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT5NGDmZB2tG_r_sv1pY71FY6mcwpBSbNR8_-BugNJEJ9jodMRg7gr-64IL0rsEhod9hTk&usqp=CAU)everyday power

#Retrieval-augmented Generation, or RAG for short.

"Check the model with a HuggingFace pipeline. Check the model with a HF pipeline, using a query about the meaning of State of the Union (SOTU).

In [9]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain `Why is a raven like a writing-desk?' Give just a definition. Keep it in 100 words.")

"\n\nAnswer:\nA raven like a writing-desk? Why, of course! It's because both are symbols of mystery and the sublime. The raven represents the unknown and the unconscious, while the writing-desk represents the creative process and the pursuit of knowledge. Together, they form a perfect pair, reminding us that the mysteries of the mind are just as important as the mysteries of the universe."

#Wow! Even myself couldn't provide that "Misterious" explanation about the Raven & Writing-Desk 

In [10]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

loader = TextLoader("/kaggle/input/alice-wonderland-dataset/alice_in_wonderland.txt",
                    encoding="utf8")
documents = loader.load()

#Split data in chunks

We split data in chunks using a recursive character text splitter.

In [11]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [12]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [13]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [14]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

#Test RAG

In [15]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/

def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQnTwExzKa2f1kIoei74b2RiA6D-Av0Cb2y6c4FnMfsTh1Vi3ncwZHmUCEqljkUz_-1i1w&usqp=CAU)yourtango

#Drawing of a Muchness???

"Once upon a time there were three little sisters, the Dormouse began in a great hurry; and their names were Elsie, Lacie, and Tillie; and they lived at the bottom of a well."

"They were learning to draw,' the Dormouse went on, yawning and rubbing its eyes, for it was getting very sleepy; and they drew all manner of things--everything that begins with an M--"

The Dormouse had closed its eyes by this time, and was going off into a doze; but, on being pinched by the Hatter, it woke up again with a little shriek, and went on: that begins with an M, such as mouse-traps, and the moon, and memory, and muchness-- you know you say things are "much of a muchness"--did you ever see such a thing as a drawing of a muchness?'

https://www.kaggle.com/datasets/roblexnana/alice-wonderland-dataset

In [16]:
query = "Why the three little sister drew all manner of things--everything that begins with an M? Summarize. Keep it under 200 words."
test_rag(qa, query)

Query: Why the three little sister drew all manner of things--everything that begins with an M? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 10.599 sec.

Result:   The three little sisters in the story drew all manner of things that began with the letter M, including mouse-traps, the moon, memory, and muchness. The Dormouse explained that they were learning to draw and were imagining all sorts of things, including the strange creatures of their little sister's dream. Alice was confused and asked why they drew these things, but the Dormouse and the Hatter were not interested in answering her question.


![](https://lonestarart.com/cdn/shop/products/tin-metal-sign-off-with-her-head-off-with-everyones-head-8x1212x18use-indooroutdoor-for-alice-in-wonderland-fans-lone-star-art-104679_533x.jpg?v=1653665829)

In [17]:
query = "What's the Queen of Hearts often shouting? Summarize. Keep it under 200 words."
test_rag(qa, query)

Query: What's the Queen of Hearts often shouting? Summarize. Keep it under 200 words.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 6.422 sec.

Result:   The Queen of Hearts is often shouting "Off with her head!" This is a common phrase used by the Queen to indicate that she wants someone to be executed by having their head cut off.


![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRg0SjQviLmfvSPQgj60F6S_qcbKrkEgw7stjn-baTNV9Mwuxl0JOYF1aNeILBEg1Fv37A&usqp=CAU)crafsuprint

In [18]:
query = "Who was the author of Alice's Adventures in Wonderland ? Keep it in 200 words."
test_rag(qa, query)

Query: Who was the author of Alice's Adventures in Wonderland ? Keep it in 200 words.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 4.531 sec.

Result:   The author of Alice's Adventures in Wonderland is Lewis Carroll.


#At the end, I can shout: OFF with the GPU! 

#Mission Accomplished in the Wonderland of LLMs. 

#Congratulations RAG, Llama2, Langchain and ChromaDB!

#Acknowledgements:

Gabriel Preda https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb/